## Lets Create a Price Adjuster (Deep Network)

Alright, so now let us see if we can come up with a deep network solution and see how it
compares to the K Nearest Neighbors we just implemented.  

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

from pprint import pprint

Lets start by loading and adjusting our data like we did in our last tutorial.  

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('../data/listing.csv')

columns = ['bathrooms', 'bedrooms', 'beds', 'accommodates', 'price']
data.dropna(axis=0, how='any', subset=columns, inplace=True)

data['price'] = data['price'].str.replace('$', '')
data['price'] = data['price'].str.replace(',', '')
data['price'] = data['price'].astype(float)

data = data[columns]

train_set, test_set = train_test_split(data)

So the first thing we are going to do is to define our inputs into the network.  For our
current setup we are going to use continuous values.  

In [ ]:
## Lets start by defining our initial network....

input_count = len(columns)-1
output_count = 1
hidden_size = 100  # This is a hyper parameter. 

In [ ]:
## Now we need the input and output placeholders in tensorflow

X = tf.placeholder(tf.float32, [None, input_count])
Y = tf.placeholder(tf.float32, [None, output_count])

In [ ]:
## There are a couple of other variables that we need

epochs = 5
batch_size = 128
learning_rate = 0.01

In [ ]:
## At this point we need to create our weights and biases for the layer connections

weights = {
    'hidden': tf.Variable(tf.random_normal([input_count, hidden_size])),
    'output': tf.Variable(tf.random_normal([hidden_size, output_count])),
}

biases = {
    'hidden': tf.Variable(tf.random_normal([hidden_size])),
    'output': tf.Variable(tf.random_normal([output_count])),
}

In [ ]:
## So now we can create out network

hidden_layer = tf.add(tf.matmul(X, weights['hidden']), biases['hidden'])
hidden_layer = tf.nn.relu(hidden_layer)
output_layer = tf.matmul(hidden_layer, weights['output']) + biases['output']

In [ ]:
## Lets define the cost function that our network will use to use with accuracy

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=Y))
loss = tf.reduce_sum(tf.square(Y - output_layer))

In [ ]:
## At this point we can teach it how to 'learn' (basically which optimizer it should use)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [ ]:
## So lets first initialize all the variables

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    label_set = train_set['price'].values.reshape(-1, output_count)
    #batch_set = train_set.drop('price', 1).reshape(-1, input_num_units)
    batch_set = train_set.drop('price', 1)
    sess.run([optimizer, loss], feed_dict = {X: batch_set, Y: label_set})

    pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, 'float'))
    
    print('Validation Accuracy: {}'.format(accuracy.eval({X: batch_set, Y: label_set})))    
    #print "Validation Accuracy:", accuracy.eval({x: val_x.reshape(-1, input_count), y: dense_to_one_hot(val_y)})

    test_label_set = test_set['price'].values.reshape(-1, output_count)
    test_batch_set = test_set.drop('price', 1)
    
    predict = tf.argmax(output_layer, 1)
    pred = predict.eval({X: test_batch_set})
    
    print(pred)
